In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain.schema.runnable import RunnablePassthrough, RunnableSequence, RunnableParallel, RunnableLambda, RunnableBranch

groq_api_key=os.getenv("GROQ_API_KEY")

model= ChatGroq(groq_api_key=groq_api_key,model_name="qwen-2.5-32b")
model2= ChatGroq(groq_api_key=groq_api_key,model_name="llama3-8b-8192")

In [7]:
prompt = PromptTemplate(
    template='Generate a details report about {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarize the following text \n{text}',
    input_variables=['text']
)

In [3]:
## Runnable Lambda
def word_counter(text):
    return len(text.split())

In [4]:
parser = StrOutputParser()

In [8]:
report_gen_chain = RunnableSequence(prompt, model, parser)

In [16]:
branch_chain = RunnableBranch(
    (lambda x: len(x.split()) > 100 , RunnableSequence(prompt2, model2, parser)),
    RunnablePassthrough()
)

In [17]:
final_chain = RunnableSequence(report_gen_chain, branch_chain)

In [18]:
final_chain.get_graph().print_ascii()

  +-------------+    
  | PromptInput |    
  +-------------+    
          *          
          *          
          *          
+----------------+   
| PromptTemplate |   
+----------------+   
          *          
          *          
          *          
    +----------+     
    | ChatGroq |     
    +----------+     
          *          
          *          
          *          
+-----------------+  
| StrOutputParser |  
+-----------------+  
          *          
          *          
          *          
    +--------+       
    | Branch |       
    +--------+       
          *          
          *          
          *          
  +--------------+   
  | BranchOutput |   
  +--------------+   


In [19]:
response = final_chain.invoke(
    {
        'topic': "Machine Learning"
    }
)

In [20]:
print(response)

Here is a summary of the text:

Machine Learning (ML) is a subset of Artificial Intelligence (AI) that enables computers to perform tasks without explicit instructions by relying on patterns and inference. ML has revolutionized various industries, including healthcare, finance, transportation, and entertainment.

The text covers the following core concepts and definitions:

* Supervised Learning: training a model on a dataset with input and output variables
* Unsupervised Learning: training a model on data without labeled responses
* Reinforcement Learning: training a model to make decisions through trial and error

Key techniques and algorithms mentioned include:

* Linear Regression
* Decision Trees
* Support Vector Machines (SVM)
* Neural Networks
* Random Forest

Applications of ML include:

* Healthcare: predicting patient diagnoses, drug discovery, and personalized medicine
* Finance: fraud detection, credit scoring, and algorithmic trading
* Marketing: personalized marketing, cu